## Deliverable 2. Create a Customer Travel Destinations Map.

In [31]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [32]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Kapaa,22.08,-159.32,78.8,78,20,16.11,US,few clouds
1,1,Albany,42.60,-73.97,73.0,38,0,1.83,US,clear sky
2,2,Ushuaia,-54.80,-68.30,53.6,48,40,25.28,AR,scattered clouds
3,3,Vila Franca Do Campo,37.72,-25.43,59.0,87,40,13.87,PT,shower rain
4,4,Duluth,46.78,-92.11,71.6,43,1,17.22,US,clear sky


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Kapaa,22.08,-159.32,78.80,78,20,16.11,US,few clouds
8,8,Samarai,-10.62,150.67,79.86,84,92,17.11,PG,moderate rain
9,9,Porto Belo,-27.16,-48.55,78.01,69,20,14.99,BR,few clouds
12,12,Avarua,-21.21,-159.78,77.00,73,85,8.59,CK,overcast clouds
13,13,Vila Velha,-20.33,-40.29,84.20,74,20,17.22,BR,few clouds
15,15,Bairiki,1.33,172.98,80.60,74,40,5.82,KI,scattered clouds
17,17,Chichihualco,17.68,-99.65,79.50,54,90,2.95,MX,overcast clouds
20,20,Salalah,17.02,54.09,80.60,74,0,4.70,OM,clear sky
25,25,Sabang,5.89,95.32,81.18,75,99,2.93,ID,overcast clouds
26,26,Tamiahua,21.27,-97.45,80.11,67,87,5.77,MX,overcast clouds


In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                212
City                   212
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Country                212
Current Description    212
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Kapaa,22.08,-159.32,78.80,78,20,16.11,US,few clouds
8,8,Samarai,-10.62,150.67,79.86,84,92,17.11,PG,moderate rain
9,9,Porto Belo,-27.16,-48.55,78.01,69,20,14.99,BR,few clouds
12,12,Avarua,-21.21,-159.78,77.00,73,85,8.59,CK,overcast clouds
13,13,Vila Velha,-20.33,-40.29,84.20,74,20,17.22,BR,few clouds
...,...,...,...,...,...,...,...,...,...,...
719,719,Inhambane,-23.86,35.38,80.60,88,40,13.87,MZ,scattered clouds
720,720,Caravelas,-17.71,-39.25,79.99,72,99,8.68,BR,overcast clouds
722,722,Dabola,10.75,-11.12,76.69,36,0,1.90,GN,clear sky
723,723,Pemangkat,1.17,108.97,79.70,81,100,2.57,ID,heavy intensity rain


In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,78.80,few clouds,22.08,-159.32,
8,Samarai,PG,79.86,moderate rain,-10.62,150.67,
9,Porto Belo,BR,78.01,few clouds,-27.16,-48.55,
12,Avarua,CK,77.00,overcast clouds,-21.21,-159.78,
13,Vila Velha,BR,84.20,few clouds,-20.33,-40.29,
15,Bairiki,KI,80.60,scattered clouds,1.33,172.98,
17,Chichihualco,MX,79.50,overcast clouds,17.68,-99.65,
20,Salalah,OM,80.60,clear sky,17.02,54.09,
25,Sabang,ID,81.18,overcast clouds,5.89,95.32,
26,Tamiahua,MX,80.11,overcast clouds,21.27,-97.45,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
#     except Exception as ex:
#         print(ex)
        
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,78.80,few clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
8,Samarai,PG,79.86,moderate rain,-10.62,150.67,Nuli Sapi
9,Porto Belo,BR,78.01,few clouds,-27.16,-48.55,Costa Verde Beach Hotel
12,Avarua,CK,77.00,overcast clouds,-21.21,-159.78,Paradise Inn
13,Vila Velha,BR,84.20,few clouds,-20.33,-40.29,Hotel Vitória Palace
15,Bairiki,KI,80.60,scattered clouds,1.33,172.98,Tarawa Boutique Hotel
17,Chichihualco,MX,79.50,overcast clouds,17.68,-99.65,Hotel Casa Blanca
20,Salalah,OM,80.60,clear sky,17.02,54.09,HAMDAN PLAZA HOTEL
25,Sabang,ID,81.18,overcast clouds,5.89,95.32,Hotel Citra
26,Tamiahua,MX,80.11,overcast clouds,21.27,-97.45,Mission Of The Sun Hotel


In [42]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,78.80,few clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
8,Samarai,PG,79.86,moderate rain,-10.62,150.67,Nuli Sapi
9,Porto Belo,BR,78.01,few clouds,-27.16,-48.55,Costa Verde Beach Hotel
12,Avarua,CK,77.00,overcast clouds,-21.21,-159.78,Paradise Inn
13,Vila Velha,BR,84.20,few clouds,-20.33,-40.29,Hotel Vitória Palace
15,Bairiki,KI,80.60,scattered clouds,1.33,172.98,Tarawa Boutique Hotel
17,Chichihualco,MX,79.50,overcast clouds,17.68,-99.65,Hotel Casa Blanca
20,Salalah,OM,80.60,clear sky,17.02,54.09,HAMDAN PLAZA HOTEL
25,Sabang,ID,81.18,overcast clouds,5.89,95.32,Hotel Citra
26,Tamiahua,MX,80.11,overcast clouds,21.27,-97.45,Mission Of The Sun Hotel


In [44]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [47]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))